In [ ]:
!pip install imbalanced-learn
!pip install delayed


     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 226 kB 27.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install imbalanced-learn
#!pip install delayed

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import sklearn
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# path of dataset
data_path = "./drive/MyDrive/Colab Notebooks/data/Channel Detection Dataset/Training Dataset/"
data_arrays_path = "./drive/MyDrive/Colab Notebooks/data_arrays/Channel Detection Dataset/Training Dataset/"

# assigning parameters for MFCC feature extraction
max_len_nfft = 345
n_mfcc = 128

In [ ]:
# Saving data to array file
save_data_to_array(data_path=data_path, data_arrays_path=data_arrays_path, max_len=max_len_nfft, n_mfcc=n_mfcc)

Saving vectors of label - 'Cartoon Network': 100%|██████████| 860/860 [00:37<00:00, 22.74it/s]
Saving vectors of label - 'ChennelDetection Negative': 100%|██████████| 480/480 [00:19<00:00, 24.37it/s]
Saving vectors of label - 'HBO': 100%|██████████| 1220/1220 [00:56<00:00, 21.77it/s]


In [ ]:
# Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test(data_path=data_path, data_arrays_path=data_arrays_path, split_ratio=0.8)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

1 ChennelDetection Negative
2 Discovery
3 Geo News
4 Saama News
5 HBO
X_train:  (4128, 128, 345)
X_test:  (1032, 128, 345)


In [ ]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# reshaping the data for oversampling
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

# fit and apply the transform
X_train, y_train = oversample.fit_resample(X_train, y_train)
X_test, y_test = oversample.fit_resample(X_test, y_test)

# converting it back to original shape
X_train = X_train.reshape(X_train.shape[0], n_mfcc, max_len_nfft)
X_test = X_test.reshape(X_test.shape[0], n_mfcc, max_len_nfft)

print("X_train: ", X_train.shape, end='\t')
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape, end='\t')
print("y_test: ", y_test.shape)

X_train:  (4698, 128, 345)	y_train:  (4698,)
X_test:  (1202, 128, 345)	y_test:  (1202,)


In [ ]:
# Feature dimension
epochs = 200
batch_size = 32
num_classes = 6

# model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(n_mfcc, max_len_nfft, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 344, 16)      80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 172, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 172, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 171, 32)       2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 85, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 85, 32)        0

In [ ]:
model.compile(loss="SparseCategoricalCrossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [ ]:
# model.fit(X_train, y_train_hot, epochs=epochs, validation_data=(X_test, y_test_hot))
model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/50
147/147 [==============================] - 19s 48ms/step - loss: 1.8710 - accuracy: 0.2431 - val_loss: 1.7133 - val_accuracy: 0.3511
Epoch 2/50
147/147 [==============================] - 6s 42ms/step - loss: 1.6278 - accuracy: 0.3284 - val_loss: 1.8251 - val_accuracy: 0.1955
Epoch 3/50
147/147 [==============================] - 6s 42ms/step - loss: 1.5748 - accuracy: 0.3382 - val_loss: 1.6423 - val_accuracy: 0.3303
Epoch 4/50
147/147 [==============================] - 6s 43ms/step - loss: 1.5428 - accuracy: 0.3523 - val_loss: 1.5865 - val_accuracy: 0.3353
Epoch 5/50
147/147 [==============================] - 6s 42ms/step - loss: 1.4938 - accuracy: 0.3880 - val_loss: 1.5771 - val_accuracy: 0.3245
Epoch 6/50
147/147 [==============================] - 6s 43ms/step - loss: 1.4600 - accuracy: 0.4066 - val_loss: 1.4371 - val_accuracy: 0.4168
Epoch 7/50
147/147 [==============================] - 6s 43ms/step - loss: 1.4361 - accuracy: 0.4163 - val_loss: 1.4998 - val_accuracy: 0.412

In [ ]:
model.save("./Models/Channel Detection Model")

INFO:tensorflow:Assets written to: ./Models/Channel Detection Model/assets


In [ ]:
model.save("./drive/MyDrive/Colab Notebooks/Models/Channel Detection Model")

INFO:tensorflow:Assets written to: ./drive/MyDrive/Colab Notebooks/Models/Channel Detection Model/assets


In [ ]:
mfcc_vectors = []
wavfiles = [data_path + "Geo News" + '/' + wavfile for wavfile in os.listdir(data_path + '/' + "Geo News")]
for wavfile in wavfiles:
    mfcc = wav2mfcc(wavfile, nfft_array=[], max_len=605, n_mfcc=128)
    mfcc_vectors.append(mfcc)

In [ ]:
y = []
j = 0
print(len(mfcc_vectors))
for i in range(len(mfcc_vectors)):
    test_point = mfcc_vectors[i].reshape(1, 128, 605)
    result = model.predict(test_point)
    result = result.argmax(axis=1)
    if result == 3:
      j += 1
print(j)

120
115


0  Saama News , 1  Masala TV , 2  Q-Tv ,3 Geo News, 4 Hum TV , 5 Discovery ,6 

1.   List item
2.   List item

ARY DIGITAL, 7 HBO , 8 ARY News , 9 Cartoon Network

In [ ]:
test_data = wav2mfcc("/content/Ruha tv par HBO lagao.wav", [], 128, 605)
test_data = test_data.reshape(1, 128, 605)
result = model.predict(test_data)
print(result)
result = result.argmax(axis=1)
print(result)

[[1.4065574e-06 1.7452343e-05 5.1558185e-03 2.0708540e-01 4.5694271e-04
  5.6294398e-03 2.0361276e-02 7.6109499e-01 1.8825750e-04 8.9496443e-06]]
[7]
